## ADAPTED APPLICATION TEMPLATE E-OBS - IN-SITU AIR TEMPERATURE

Package c3s_eqc_automatic_quality_control (https://github.com/bopen/c3s-eqc-automatic-quality-control).
This notebook is a first test for the implementation on the following diagnostics on the 
E-Obs in-situ gridded dataset, chosing as variable of interest Air Temperature:
- Climatology Maps for the variable;
- Time Series of Globally Averaged variable;
As initial test to carry out the diagnostic proposed in the deliverable submitted on September 2022  C3S2_D520.5.1.6_202209_Consolidated_diagnostics_in-situ_v1 
Which can be found in the +Atlantic Teams folder 
WP5 EQC of Reanalysis, Satellite and In-situ Obsv > 1_Deliverables > 09_2022 > 2_Submitted

First we load the necessary modules and set up the option to load in the cache the transformed data:

In [ ]:
import warnings
import plotly.express as px
import xarray as xr
import cads_toolbox

from c3s_eqc_automatic_quality_control import diagnostics, download, plot

warnings.filterwarnings("ignore")
cads_toolbox.config.USE_CACHE = True

Then, we create our request to download data and perform our analyses

In [ ]:
collection_id = "insitu-gridded-observations-europe"
request = {
    "product_type": "ensemble_mean",
    "variable": "mean_temperature",
    "grid_resolution": "0.25deg",
    "period": ["1950_1964", "1965_1979", "1980_1994", "1995_2010", "2011_2021"],
    "version": "25.0e",
    "format": "zip",
}
request

We define:
- an identity function to download data as it is without performing any operation.
- A function to calculate the spatial mean and aggregate the data into a monthly mean temporal resolution;
- A function to get the monthly climatology from a dataset;

In [ ]:
def identity(ds: xr.Dataset) -> xr.Dataset:
    return ds
def spatial_monthly_mean(ds: xr.Dataset) -> xr.Dataset:
    ds = diagnostics.spatial_weighted_mean(ds)
    return ds.resample(time="1M").mean("time")
def monthly_climatology(ds: xr.Dataset) -> xr.Dataset:
    ds_month = ds.groupby('time.month').mean('time')
    return ds_month

Now we retrieve the twelve climatology maps downloading and transforming our raw data from the Climate and Atmosphere Data Store (CADS in short). Since the result of such operation will be splitted in chunks which contain respectively one of the items in the list given to the "period" keyword in the request of above, we'll need to apply the mean over the dimension 'time' to get just a single climatology field composed of 12 snapshot, each for one month of the dataset:

In [ ]:
monthlyclim_temp_maps = download.download_and_transform(
    collection_id,
    request,
    transform_func=monthly_climatology,
    split_all=True,
    concat_dim='time',
    combine='nested'
)
monthlyclim_temp_map = monthlyclim_temp_maps.mean('time')

Now we re-define a spatial mean function which does not involve any resampling of the data, to get the climatology of the timeseries of globally averaged temperature field:

In [ ]:
def spatial_mean(ds: xr.Dataset) -> xr.Dataset:
    ds = diagnostics.spatial_weighted_mean(ds)
    return ds
monthly_global_temp_ts = spatial_mean(monthlyclim_temp_map)
monthly_global_temp_ts

We then just call the function in plotly.express to plot the result of the above operation, storing the image into a file of a given name.

In [ ]:
fig = px.line(
    x=monthly_global_temp_ts["month"],
    y=monthly_global_temp_ts["tg"],
)
fig.update_layout(
    xaxis_title=r"$\mbox{Time [months]}$",
    yaxis_title=r"$\mbox{Temperature}~[^oC]$",
    title="GLOBAL MEAN MONTHLY CLIMATOLOGY TEMPERATURE 1950-2021",
)
fig.show()

In [ ]:
fig.write_image('E-Obs_spavg_monthlyclim_1950-2021.png')

### Now the monthly climatology maps
We now create a variable var to plot the climatology maps, in such a way that it will be possible to create a function to plot spatial maps which will eventually be inserted in the plotting methods of the automatic quality check package or the toolbox itself:

In [ ]:
clim_ds = monthlyclim_temp_map
var = clim_ds['tg']
m = int((var.min()/10).round(2))*10
M = int((var.max()/10).round(2))*10
print(m, M)
var

We create an interactive map with plotly to have contour plots for each month of our climatology:

In [ ]:
import plotly.graph_objects as go
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
i=0
for i in range(len(var.month)):
    fig = go.Figure(data =
                    go.Contour(
                        x = var.longitude,
                        y = var.latitude,
                        z = var.isel(month=i).values,
                        colorscale='jet',
                        colorbar=dict(
                            title='°C', # title here
                            titleside='right',
                            titlefont=dict(
                                size=14,
                                family='Arial, sans-serif')
                        ),
                        contours=dict(
                            start=m,
                            end=M,
                            size=1,
    )))
    fig.update_layout(
        title={
            'text': r'TEMPERATURE '+months[i]+' Climatology 1950-2021',
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_title='longitude [degrees_east]',
        yaxis_title='latitude [degrees_north]',
    )
    fig.write_image('E-OBS_climatology_map_temp_1950-2021_'+months[i]+'.png')
    fig.show()